# Experiment 2 - Process tactile image data
* processed tactile video data saved into two arrays:
*        feature_SA and features_RA of shape (n_pins, n_pins)
* n_pins is number of pins along side of square array (here 19)
* values of features are the SA and FA firing rates from module afferent_response.py
* the features are normalised to mean 0 stdev 1 on each individual data set
* each dataset comprises frames at nxs distinct poses, from which yaw (last component) is used as a label
* dataset processing is iterated as a list over (1) test/train datasets (2) 8 grating stimuli (3) 4 grating orientations (physically moved)

To run, first edit dir_data to path where data stored.

In [1]:
import os, sys, json, pickle
import numpy as np
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath('../common'))
from afferent_response import AfferentResponse as ar

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [2]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-gratings-jrsi2022/experiment_2/"
datasets = ['train_data','test_data']
n_stimuli = 8
n_angles = 4
stimuli = [f'{i}' for i in range(n_stimuli)]
angles = [f'{90*a}' for a in range(n_angles)]
n_pins = 19

In [3]:
with open(r"../common/tip_params/tipA", 'r') as read_file: 
    tip_params = json.load(read_file)
afferents = ar(False, **tip_params)
pins_init = afferents.init_pins(dir_data + r"/train_data/throw_away.mp4")
print(f"n_pins={len(pins_init)}")

n_pins=361


In [4]:
def rotate_pins(pins, theta):
    R = np.array(((np.cos(theta), -np.sin(theta)), (np.sin(theta), np.cos(theta))))
    pins_rot = [np.matmul(R, pin) for pin in pins]
    return np.array(pins_rot)

def locate_pins(pins, n_pins):
    locs = np.zeros((n_pins, n_pins))
    index_xSort = np.argsort(pins[:,0], axis=0)
    
    index_ySort = np.zeros((n_pins, n_pins))
    n = 0
    for k in range(n_pins):
        indices = np.zeros((n_pins,2))
        for l in range(n_pins):
            indices[l,:] = pins[index_xSort[n],:]
            n+=1
        index_ySort[k,:] = np.argsort(indices[:,1], axis=0)

    for k in range(n_pins):
        for l in range(n_pins):
            locs[k,l] = index_xSort[int(index_ySort[k,l])+n_pins*k]
    return locs

def features(firing, locs):
    n1_pins, n2_pins = np.shape(locs)
    features = np.zeros((n1_pins, n2_pins))
    for k in range(n1_pins):
        for l in range(n2_pins):
            features[k,l] = firing[int(locs[k,l])]     
    return features

In [8]:
for dataset in datasets:
    if dataset=='train_data': nxs = 250
    if dataset=='test_data': nxs = 150
    
    for stimulus in stimuli:
        for angle in angles:
            in_dir = dir_data + rf'/{dataset}/{stimulus}/{angle}/'
            out_dir = dir_data + rf'/processed/{dataset}/{stimulus}/{angle}/'
            print(in_dir, out_dir)

            rxs = [1] + list(range(1,nxs)) # hack: not using the first tap because out of focus
            firing = [afferents.firing(in_dir + rf"/videos/x_{i}.mp4", pins_init) for i in rxs]
        
            firing_SA, firing_RA, pins = [[] for _ in range(3)]
            for i in range(nxs):
                nts = len(firing[i][0])
                if nts>0:
                    t_maxSA = np.argmax([np.sum(firing[i][0][f'{t}']) for t in range(nts)])
                    t_maxRA = np.argmax([np.sum(firing[i][1][f'{t}']) for t in range(nts)])
                    firing_SA.append(firing[i][0][f'{t_maxSA}'])
                    firing_RA.append(firing[i][1][f'{t_maxRA}'])
                    pins.append(firing[i][2])
                else:
                    firing_SA.append(firing_SA[i-1])
                    firing_RA.append(firing_RA[i-1])
                    pins.append(pins[i-1]) 
        
            # Rotate pin positions and sort features as firing
            pins_rot = [rotate_pins(pins[i], np.radians(-6.2)) for i in range(nxs)]
            locs_rot = [locate_pins(pins_rot[i], n_pins) for i in range(nxs)]
            features_SA = [features(firing_SA[i], locs_rot[i]) for i in range(nxs)]
            features_RA = [features(firing_RA[i], locs_rot[i]) for i in range(nxs)]
       
            # normalise each image independently to have a mean of zero and a std dev of 1
            features_SA = [(features_SA[i]-features_SA[i].mean())/features_SA[i].std() for i in range(nxs)]
            features_RA = [(features_RA[i]-features_RA[i].mean())/features_RA[i].std() for i in range(nxs)]
                        
            # label with yaw
            with open(in_dir + r"/metadata.json", 'r') as read_file:
                metadata = json.load(read_file)
            labels = [np.asarray(metadata['xs'])[j,5] for j in rxs]
        
            os.makedirs(out_dir, exist_ok=True)

            if dataset=='train_data':
                X_train_SA, X_val_SA, y_train_SA, y_val_SA = train_test_split(features_SA, labels, test_size=0.25, random_state=42)
                X_train_RA, X_val_RA, y_train_RA, y_val_RA = train_test_split(features_RA, labels, test_size=0.25, random_state=42)

                save_obj(X_train_SA, out_dir + r"/X_train_sa")
                save_obj(X_train_RA, out_dir + r"/X_train_ra")
                save_obj(y_train_SA, out_dir + r"/y_train_sa")
                save_obj(y_train_RA, out_dir + r"/y_train_ra")
                save_obj(X_val_SA, out_dir + r"/X_val_sa")
                save_obj(X_val_RA, out_dir + r"/X_val_ra")
                save_obj(y_val_SA, out_dir + r"/y_val_sa")
                save_obj(y_val_RA, out_dir + r"/y_val_ra")

            if dataset=='test_data':
                save_obj(features_SA, out_dir + r"/X_test_sa")
                save_obj(features_RA, out_dir + r"/X_test_ra")
                save_obj(labels, out_dir + r"/y_test_sa")
                save_obj(labels, out_dir + r"/y_test_ra")

C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/shared/nick_pestell/JRSI_gratings/experiment_2//train_data/0/0/ C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/shared/nick_pestell/JRSI_gratings/experiment_2//processed/train_data/0/0/
C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/shared/nick_pestell/JRSI_gratings/experiment_2//train_data/0/90/ C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/shared/nick_pestell/JRSI_gratings/experiment_2//processed/train_data/0/90/
C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/shared/nick_pestell/JRSI_gratings/experiment_2//train_data/0/180/ C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/shared/nick_pestell/JRSI_gratings/experiment_2//processed/train_data/0/180/
C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/shared/nick_pestell/JRSI_gratings/experiment_2//train_data/0/270/ C:\Users\nl13426\OneDrive - University of Bristol\Data onedrive/sh